# 01. 개발환경 설정하기

+ [Google Cloud Speech-to-Text 개발환경 설정하기](https://github.com/sungalex/VoiceMagic/blob/master/google%20cloud%20speech-to-text%20%EA%B0%9C%EB%B0%9C%ED%99%98%EA%B2%BD%20%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0.pdf)
+ [Google Cloud Speech-to-Text 참고자료](https://github.com/sungalex/VoiceMagic)


# 02. 파이썬 라이브러리 설치

In [ ]:
pip install --upgrade google-cloud-storage # 구글 클라우드 플랫폼에서 버킷으로 음성파일 분석할 때 필요 

In [ ]:
pip install google-cloud-speech

In [ ]:
pip install --upgrade google-cloud-speech

In [ ]:
pip install pyaudio # (Window, Mac) 음성인식 기능 패키지 pyaudio 설치

#### (For Mac) pip install pyaudio 에러 발생 시 터미널 킨 후 아래와 같이 시도
    $ brew update
$ brew install portaudio
    $ brew link --overwrite portaudio
$ pip install pyaudio 

# 03. 환경변수 설정
+ 주피터 노트북에서 Google STT api를 실행하기 위해 json파일 환경변수 설정 필요함
+ 구글 클라우드 플랫폼에서 비공개 json 파일을 다운받은 후, 아래 경로에 json파일 경로를 입력
+ (For Mac) 파일 경로를 알기 힘들 경우, 터미널을 킨 후 파일을 터미널에 드래그하면 경로 출력됨

In [ ]:
import os
    # 환경변수 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/hyunwoo/Desktop/speech-to-text-273518-4131322696e3.json"

## Microphone Streaming emulate

In [ ]:
from __future__ import division

import re
import sys

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import pyaudio
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)

def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if response.speech_event_type:
            text = transcript + overwrite_chars
            print('here is final text: {}'.format(text))
            return text
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = ' ' * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + '\r')
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r'\b(exit|quit)\b', transcript, re.I):
                print('Exiting..')
                break

            num_chars_printed = 0

def main():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = 'ko-KR'  # a BCP-47 language tag  'en-US' or 'ko-KR'

    client = speech.SpeechClient()
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code)
    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        single_utterance=True,
        interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (types.StreamingRecognizeRequest(audio_content=content)
                    for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)

if __name__ == '__main__':
    main()

#  기타

## Google Storage bucket으로 음성파일을 텍스트 변환

In [ ]:
from google.cloud import speech_v1p1beta1
from google.cloud.speech_v1p1beta1 import enums

def sample_recognize(storage_uri):
    """
    Performs synchronous speech recognition on an audio file

    Args:
        storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """
    
    client = speech_v1p1beta1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.mp3'

    # The language of the supplied audio
    language_code = "ko-KR"  # ko-KR, en-US

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.MP3
    config = {
            "language_code": language_code,
            "sample_rate_hertz": sample_rate_hertz,
            "encoding": encoding,
        }
    audio = {"uri": storage_uri}

    response = client.recognize(config, audio)
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))

sample_recognize('gs://speechtotext_bucket01/test.mp3')

In [ ]:
def explicit():
        from google.cloud import storage

        # Explicitly use service account credentials by specifying the private key
        # file.
        storage_client = storage.Client.from_service_account_json(
            '/Users/hyunwoo/Desktop/speech-to-text-273518-4131322696e3.json')

        # Make an authenticated API request
        buckets = list(storage_client.list_buckets())
        print(buckets)

In [ ]:
pip install --upgrade google-cloud-bigquery  # OAuth 인증을 위해 설치 (필요없어진 듯)

In [ ]:
pip install --upgrade google-auth-oauthlib # OAuth 인증을 위해 설치 (필요없어진 듯)

In [ ]:
from google.cloud import bigquery

    # OAuth를 위한 코드

    # TODO: Uncomment the line below to set the `project` variable.
    # project = 'user-project-id'
    #
    # The `project` variable defines the project to be billed for query
    # processing. The user must have the bigquery.jobs.create permission on
    # this project to run a query. See:
    # https://cloud.google.com/bigquery/docs/access-control#permissions

client = bigquery.Client(project='speech-to-text-273518', credentials=credentials)

query_string = """SELECT name, SUM(number) as total
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE name = 'William'
GROUP BY name;
"""
query_job = client.query(query_string)

# Print the results.
for row in query_job.result():  # Wait for the job to complete.
    print("{}: {}".format(row['name'], row['total']))

In [ ]:
from google_auth_oauthlib import flow

    # OAuth 인증을 위한 코드

    # TODO: Uncomment the line below to set the `launch_browser` variable.
    # launch_browser = True
    #
    # The `launch_browser` boolean variable indicates if a local server is used
    # as the callback URL in the auth flow. A value of `True` is recommended,
    # but a local server does not work if accessing the application remotely,
    # such as over SSH or from a remote Jupyter notebook.

appflow = flow.InstalledAppFlow.from_client_secrets_file(
        '/Users/hyunwoo/Desktop/client_secret_903962781159-e09epgnjjf53dq0bsevc7t90m6esj79c.apps.googleusercontent.com.json',
scopes=['https://www.googleapis.com/auth/bigquery'])
    
launch_browser = True

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()

credentials = appflow.credentials